In [1]:
import gensim.downloader as api

In [2]:
model = api.load('word2vec-google-news-300')

In [3]:
model.similarity(w1='good', w2='mobile')

0.027614867

In [4]:
model.most_similar('good')

[('great', 0.7291510105133057),
 ('bad', 0.7190051078796387),
 ('terrific', 0.6889115571975708),
 ('decent', 0.6837348341941833),
 ('nice', 0.6836092472076416),
 ('excellent', 0.644292950630188),
 ('fantastic', 0.6407778263092041),
 ('better', 0.6120728850364685),
 ('solid', 0.5806034803390503),
 ('lousy', 0.576420247554779)]

In [5]:
# King - man + women = queen
model.most_similar(positive=['king','women'], negative=['men'])

[('queen', 0.6525818109512329),
 ('monarch', 0.5959680676460266),
 ('crown_prince', 0.5324338674545288),
 ('kings', 0.5282072424888611),
 ('princess', 0.5240627527236938),
 ('sultan', 0.4983791410923004),
 ('monarchy', 0.49648746848106384),
 ('ruler', 0.49376848340034485),
 ('prince', 0.48910900950431824),
 ('Prince_Paras', 0.4794894754886627)]

In [6]:
model.doesnt_match(['facebook', 'twitter', 'cat', 'google'])

'cat'

In [7]:
## mean vector actually gives mean of vectors of given text or number
model.get_mean_vector(['hello how are you'], pre_normalize=False)

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0.

In [ ]:
def preprocess_stopword(text):
    doc = nlp(text)
    filtered_test = []
    
    for token in doc:
        if token.is_punct or token.is_stop:
            continue
        filtered_token = token.lemma_.strip()
        if filtered_token:
            filtered_test.append(filtered_token)
    
    print(text)
    print(filtered_test)
    
    if filtered_test:
        return model.get_mean_vector(filtered_test)
    else:
        return None

In [ ]:
preprocess_stopword('all i am')

#  Now we do News classification

In [ ]:
import pandas as pd
import numpy as np
df = pd.read_csv('SMSSpamCollection.csv',sep='\t',names=['label','message'],on_bad_lines='skip')
df.head()

In [ ]:
df['category'] = df.label.map({
    'ham': 0,
    'spam': 1
})

In [ ]:
df.head()

In [ ]:
df.isnull().sum()

In [ ]:
df.dropna(inplace=True)

In [ ]:
df.isnull().sum()

In [ ]:
# now we sent our title column to preprocess_stopword
df['vector'] = df['message'].apply(lambda text: preprocess_stopword(text))

In [ ]:
df.head()

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(df.vector.values, df.category, test_size=0.2, stratify=df.category, random_state=2023)

In [ ]:
X_train = X_train.reshape(-1,1)
X_test = X_test.reshape(-1,1)

In [ ]:
X_train_2d = np.stack(X_train)
X_test_2d = np.stack(X_test)

In [ ]:
print(X_train_2d.shape)
print(X_test_2d.shape)

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import classification_report
gb = GradientBoostingClassifier()
y_pred = gb.predict(X_test_2d)
print(classification_report(y_test, y_pred))

In [ ]:
## this is for convert text into vector according to similarity
def preprocessing_gensim(text, doc):
    for token in doc:
        return model.similarity(w1=text, w2=token)

In [ ]:
preprocessing_gensim(('bear', 'leopard', 'mobile'), ('lion', 'tiger','snake'))